# Main Program

In [45]:
import cv2
import math
import mediapipe as mp

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Input/output paths
input_file_name = "djv_swing"
input_file_extension = '.' + "mp4"
input_path = "input_videos/" + input_file_name + input_file_extension

output_path = "output_videos/" + input_file_name + "_angles" + ".mp4"

# Load video
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) / 15

# Video writer
fourcc = cv2.VideoWriter_fourcc(*'avc1')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Your own connections and key points
# Define custom skeleton connections (includes hips to shoulders)
custom_connections = [
    (mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.LEFT_KNEE),
    (mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_HIP),
    (mp_pose.PoseLandmark.RIGHT_ANKLE, mp_pose.PoseLandmark.RIGHT_KNEE),
    (mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_HIP),
    (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
    (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_SHOULDER),
    (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_SHOULDER),
    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
    (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
    (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
    (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
]

# Define which points to mark
red_points = [
    mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
    mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP
]

blue_points = [
    mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.RIGHT_ANKLE,
    mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.RIGHT_KNEE,
    mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.RIGHT_ELBOW,
    mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_WRIST,
    mp_pose.PoseLandmark.NOSE
]

def angle_relative_to_horizontal(joint1, joint2) -> float:
    # Anchor = joint further to the right (larger x-value)
    if joint1.x > joint2.x:
        anchor = joint1
        other = joint2
    else:
        anchor = joint2
        other = joint1

    # Reference = horizontal vector pointing left
    ref_vector = [-1.0, 0.0]
    dx = other.x - anchor.x
    dy = other.y - anchor.y
    target_vector = [dx, dy]

    dot = ref_vector[0] * target_vector[0] + ref_vector[1] * target_vector[1]
    mag_ref = math.hypot(*ref_vector)
    mag_target = math.hypot(*target_vector)

    if mag_target == 0:
        return 0.0

    angle_rad = math.acos(dot / (mag_ref * mag_target))
    angle_deg = math.degrees(angle_rad)

    # Flip sign if joint is above (y is lower)
    if dy < 0:
        angle_deg = -angle_deg

    return round(angle_deg, 1)

def draw_text_with_background(img, text, position, font, scale, text_color, thickness, bg_color=(0, 0, 0), padding=5):
    text_size, _ = cv2.getTextSize(text, font, scale, thickness)
    x, y = position
    w, h = text_size

    # Draw rectangle (filled)
    cv2.rectangle(img, (x - padding, y - h - padding), (x + w + padding, y + padding), bg_color, -1)

    # Draw text
    cv2.putText(img, text, (x, y), font, scale, text_color, thickness)

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB for MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Draw skeleton
        for start, end in custom_connections:
            p1 = landmarks[start.value]
            p2 = landmarks[end.value]
            x1, y1 = int(p1.x * width), int(p1.y * height)
            x2, y2 = int(p2.x * width), int(p2.y * height)
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw red points
        for point in red_points:
            pt = landmarks[point.value]
            x, y = int(pt.x * width), int(pt.y * height)
            cv2.circle(frame, (x, y), 6, (0, 0, 255), -1)

        # Draw blue points
        for point in blue_points:
            pt = landmarks[point.value]
            x, y = int(pt.x * width), int(pt.y * height)
            cv2.circle(frame, (x, y), 6, (255, 0, 0), -1)

        # Extract joint positions
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

        # Calculate angles
        shoulder_angle = angle_relative_to_horizontal(left_shoulder, right_shoulder)
        hip_angle = angle_relative_to_horizontal(left_hip, right_hip)

        # Label positions
        sx = int(max(left_shoulder.x, right_shoulder.x) * width)
        sy = int(min(left_shoulder.y, right_shoulder.y) * height)
        hx = int(max(left_hip.x, right_hip.x) * width)
        hy = int(min(left_hip.y, right_hip.y) * height)

        # Add text labels
        # Dynamically scale font size and thickness based on frame height
        font_scale = (height / 720) * (1/3)  # 1/3 the original size
        font_thickness = max(1, int(height / 480))  # increase thickness by 1
        
        draw_text_with_background(
            frame, f"Shoulders: {shoulder_angle} deg", (sx + 50, sy),
            cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness
        )
        
        draw_text_with_background(
            frame, f"Hips: {hip_angle} deg", (hx + 50, hy),
            cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness
        )

    # Write frame to output
    out.write(frame)

# Finalize
cap.release()
out.release()
print(f"✅ Video with side-facing joint angles saved to: {output_path}")


✅ Video with side-facing joint angles saved to: output_videos/djv_swing_angles.mp4
